## Enviroment installing

In [ ]:
!pip install opendatasets

In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


features = ['load', 'ESMode1', 'ESMode2', 'ESMode3', 'ESMode4', 'ESMode5', 'ESMode6',\
            'RUType', 'Mode', 'Frequency', 'Bandwidth', 'Antennas', 'TXpower', 'year', 'month', 'day', 'hour']

label = ['Energy']

## Data Downloading

In [ ]:
merge_data = pd.read_csv('/content/merge_minus_product.csv')

### Energy previous

In [ ]:
# Lagging value of energy

def lagging_energy(df_data):
    data_df = df_data.copy()
    for lagging in range (1, 6):
        col = 'energy_' + str(lagging)
        data_df[col] = data_df.groupby(['bs', 'hour'])['energy'].shift(lagging)
    return data_df

# 1.7651

### Energy statistical

In [ ]:
def energy_static(df_data):
    data_df = df_data.copy()
    data_df['mean_energy']  = data_df.groupby(['bs', 'hour'])['energy'].transform(lambda x: x.rolling(7, min_periods=1).mean())
    data_df['std_energy']   = data_df.groupby(['bs', 'hour'])['energy'].transform(lambda x: x.rolling(7, min_periods=1).std())
    data_df['skew_energy']  = data_df.groupby(['bs', 'hour'])['energy'].transform(lambda x: x.rolling(7, min_periods=1).skew())
    data_df['max_energy']   = data_df.groupby(['bs', 'hour'])['energy'].transform(lambda x: x.rolling(7, min_periods=1).max())
    data_df['min_energy']   = data_df.groupby(['bs', 'hour'])['energy'].transform(lambda x: x.rolling(7, min_periods=1).min())
    return data_df

# 8.056

### KPI

In [ ]:
def lagging_kpi(df_data):
    data_df = df_data.copy()
    kpis = ['load_cell0', 'load_cell1', 'load_cell2', 'load_cell3',
            'frequency_cell0', 'frequency_cell1', 'frequency_cell2', 'frequency_cell3',
            'bandwidth_cell0', 'bandwidth_cell1', 'bandwidth_cell2', 'bandwidth_cell3',
            'antennas_cell0', 'antennas_cell1', 'antennas_cell2', 'antennas_cell3',
            'txpower_cell0', 'txpower_cell1', 'txpower_cell2', 'txpower_cell3']
    # print(kpis)
    for kpi in kpis:
        for lagging in range(1, 7):
            col = kpi + '_' +str(lagging)
            data_df[col] = data_df.groupby(['bs', 'hour'])[kpi].shift(lagging)

    return data_df


# 1.845500987

### KPI difference

In [ ]:
def kpi_difference(df_data):
    data_df = df_data.copy()
    kpis = ['load_cell0', 'load_cell1', 'load_cell2', 'load_cell3',
            'frequency_cell0', 'frequency_cell1', 'frequency_cell2', 'frequency_cell3',
            'bandwidth_cell0', 'bandwidth_cell1', 'bandwidth_cell2', 'bandwidth_cell3',
            'antennas_cell0', 'antennas_cell1', 'antennas_cell2', 'antennas_cell3',
            'txpower_cell0', 'txpower_cell1', 'txpower_cell2', 'txpower_cell3']
    # print(kpis)
    for kpi in kpis:
        for lagging in range(1, 7):
            col = kpi + '_diff_' + str(lagging)
            data_df[col] = data_df.groupby(['bs', 'hour'])[kpi].diff(lagging)
    return data_df
# 1.814460556

### KPI ratio

In [ ]:
def kpi_ratio(df_data):
    data_df = df_data.copy()
    kpis = ['load_cell0', 'load_cell1', 'load_cell2', 'load_cell3',
            'frequency_cell0', 'frequency_cell1', 'frequency_cell2', 'frequency_cell3',
            'bandwidth_cell0', 'bandwidth_cell1', 'bandwidth_cell2', 'bandwidth_cell3',
            'antennas_cell0', 'antennas_cell1', 'antennas_cell2', 'antennas_cell3',
            'txpower_cell0', 'txpower_cell1', 'txpower_cell2', 'txpower_cell3']
    # print(kpis)
    for kpi in kpis:
        for lagging in range(1, 7):
            col = kpi + '_ratio_' + str(lagging)
            data_df[col] = data_df.groupby(['bs', 'hour'])[kpi].pct_change(lagging)
    return data_df

# 1.805322502

### KPI plus KPI ratio

In [ ]:
KPI_merge_KPI_ratio   = kpi_ratio(merge_data)
KPI_merge_KPI_lagging = lagging_energy(merge_data)
KPI_merge_KPI_ratio   = KPI_merge_KPI_ratio.drop(columns = ['Unnamed: 0'])
KPI_merge_KPI_lagging = KPI_merge_KPI_lagging.drop(columns = ['Unnamed: 0'])

<ipython-input-10-c452486d3be3>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[col] = data_df.groupby(['bs', 'hour'])[kpi].pct_change(lagging)
<ipython-input-10-c452486d3be3>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[col] = data_df.groupby(['bs', 'hour'])[kpi].pct_change(lagging)
<ipython-input-10-c452486d3be3>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

In [ ]:
KPI_merge_KPI_lagging.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118768 entries, 0 to 118767
Columns: 145 entries, time to energy_5
dtypes: float64(138), int64(4), object(3)
memory usage: 131.4+ MB


### Data split

In [ ]:
train_KPI_merge_KPI_ratio   = KPI_merge_KPI_ratio[KPI_merge_KPI_ratio['split'] == 'train']
test_KPI_merge_KPI_ratio    = KPI_merge_KPI_ratio[KPI_merge_KPI_ratio['split'] == 'test']

train_KPI_merge_KPI_lagging = KPI_merge_KPI_lagging[KPI_merge_KPI_lagging['split'] == 'train']
test_KPI_merge_KPI_lagging = KPI_merge_KPI_lagging[KPI_merge_KPI_lagging['split'] == 'test']

In [ ]:
train_KPI_merge_KPI_ratio.shape, test_KPI_merge_KPI_ratio.shape, train_KPI_merge_KPI_lagging.shape, test_KPI_merge_KPI_lagging.shape

((92629, 260), (26139, 260), (92629, 145), (26139, 145))

### lgb1

In [ ]:
target_col = 'energy'
drop_cols = ['time', 'bs', 'split', target_col]

In [ ]:
def feature_enginning(train_df, valid_df, test_df):


    train_df['split'] = 'train'
    valid_df['split'] = 'valid'
    test_df['split'] = 'test'

    df = pd.concat([train_df, valid_df, test_df])
    df['bs_en'] = df['bs'].apply(lambda x: int(x.strip('B_')))

    df.sort_values(['time', 'bs'], inplace=True)



    train_df = df[df['split'] =='train']
    valid_df = df[df['split'] =='valid']
    test_df = df[df['split'] =='test']
    residual_df = pd.concat([train_df, valid_df])

    return train_df, valid_df, test_df, residual_df


In [ ]:
from sklearn import metrics
from sklearn import model_selection
import lightgbm as lgb
import xgboost as xgb
import numpy as np
kf = model_selection.KFold(n_splits=5)
kf = kf.split(X=train_KPI_merge_KPI_ratio)

oof_valid_preds = np.zeros(train_KPI_merge_KPI_ratio.shape[0], )
test_preds_list = []
residual_list   = []

for i, (train_idx, valid_idx) in enumerate(kf):

    train_df = train_KPI_merge_KPI_ratio.iloc[train_idx]
    valid_df = train_KPI_merge_KPI_ratio.iloc[valid_idx]
    test_df = test_KPI_merge_KPI_ratio.copy()

    train_df, valid_df, test_df, residual_df = feature_enginning(train_df=train_df, valid_df=valid_df, test_df=test_df)
    train_cols = [col for col in train_df.columns if col not in drop_cols]

    X_train, y_train        = train_df[train_cols], train_df[target_col]
    X_valid, y_valid        = valid_df[train_cols], valid_df[target_col]
    X_test                  = test_df[train_cols]
    X_residual, y_residual  = residual_df[train_cols], residual_df[target_col]

    params = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
        # 'learning_rate': 0.01,
        'n_jobs': -1,
        'max_depth' : -1,
        # 'reg_alpha': 0.3,
        # 'reg_lambda': 0.3,
        'metric': 'mape',
        'num_boost_round': 10000,
    }

    # model = lgb.LGBMRegressor(**lgb_study.best_params)
    model1 = lgb.LGBMRegressor(**params)
    # model = lgb.LGBMRegressor()

    early_stopping_callback = lgb.early_stopping(100, first_metric_only=True, verbose=False)
    verbose_callback = lgb.log_evaluation(100)

    model1.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[early_stopping_callback, verbose_callback],
    )

    valid_preds = model1.predict(X_valid)
    test_preds = model1.predict(X_test)
    residual_preds = model1.predict(X_residual)


    val_score = metrics.mean_absolute_percentage_error(y_valid, valid_preds)
    oof_valid_preds[valid_idx] = valid_preds
    test_preds_list.append(test_preds)
    residual_list.append(residual_preds)

    print("=*"*50)
    print(f"Fold : {i}")
    print(f"Valid score : ", val_score)

oof_score = metrics.mean_absolute_percentage_error(train_KPI_merge_KPI_ratio[target_col], oof_valid_preds)
print("_-*"*50)
print(f"OOF score : ", oof_score)

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.113903 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9580
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 233
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.313705
[100]	valid_0's mape: 0.0886033
[200]	valid_0's mape: 0.0786587
[300]	valid_0's mape: 0.0748199
[400]	valid_0's mape: 0.07247
[500]	valid_0's mape: 0.0713666
[600]	valid_0's mape: 0.0701505
[700]	valid_0's mape: 0.0689787
[800]	valid_0's mape: 0.068123
[900]	valid_0's mape: 0.0677855
[1000]	valid_0's mape: 0.0672397
[1100]	valid_0's mape

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.059413 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8982
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 167
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.276547
[100]	valid_0's mape: 0.0803641
[200]	valid_0's mape: 0.0705342
[300]	valid_0's mape: 0.0670177
[400]	valid_0's mape: 0.0650252
[500]	valid_0's mape: 0.0635966
[600]	valid_0's mape: 0.0619817
[700]	valid_0's mape: 0.0612013
[800]	valid_0's mape: 0.0605081
[900]	valid_0's mape: 0.0599513
[1000]	valid_0's mape: 0.0596689
[1100]	valid_0's m

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.089822 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9661
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 233
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.059540
[100]	valid_0's mape: 0.0747463
[200]	valid_0's mape: 0.0649777
[300]	valid_0's mape: 0.0620475
[400]	valid_0's mape: 0.0596299
[500]	valid_0's mape: 0.0583992
[600]	valid_0's mape: 0.0574348
[700]	valid_0's mape: 0.0568652
[800]	valid_0's mape: 0.056212
[900]	valid_0's mape: 0.0557403
[1000]	valid_0's mape: 0.0554625
[1100]	valid_0's ma

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.086815 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9514
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 233
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 27.913267
[100]	valid_0's mape: 0.0725883
[200]	valid_0's mape: 0.0628179
[300]	valid_0's mape: 0.0591209
[400]	valid_0's mape: 0.0566503
[500]	valid_0's mape: 0.0549494
[600]	valid_0's mape: 0.0537115
[700]	valid_0's mape: 0.0531467
[800]	valid_0's mape: 0.0527181
[900]	valid_0's mape: 0.0520587
[1000]	valid_0's mape: 0.0517055
[1100]	valid_0's m

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.083416 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8710
[LightGBM] [Info] Number of data points in the train set: 74104, number of used features: 202
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.131925
[100]	valid_0's mape: 0.0793559
[200]	valid_0's mape: 0.0694022
[300]	valid_0's mape: 0.0643951
[400]	valid_0's mape: 0.0624707
[500]	valid_0's mape: 0.0613253
[600]	valid_0's mape: 0.0601656
[700]	valid_0's mape: 0.0594321
[800]	valid_0's mape: 0.0586556
[900]	valid_0's mape: 0.0582776
[1000]	valid_0's mape: 0.0579984
[1100]	valid_0's m

In [ ]:
import numpy as np
test_preds_mean = np.mean(test_preds_list, axis=0).tolist()
residual_mean   = np.mean(residual_list, axis=0).tolist()

In [ ]:
train_KPI_merge_KPI_lagging['residual'] = residual_mean
test_KPI_merge_KPI_lagging['residual'] = test_preds_mean

<ipython-input-19-81cf83925873>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_KPI_merge_KPI_lagging['residual'] = residual_mean
<ipython-input-19-81cf83925873>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_KPI_merge_KPI_lagging['residual'] = test_preds_mean


In [ ]:
train_KPI_merge_KPI_lagging.info()
train_KPI_merge_KPI_lagging.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92629 entries, 0 to 118767
Columns: 146 entries, time to residual
dtypes: float64(139), int64(4), object(3)
memory usage: 103.9+ MB


,time,bs,load_cell0,load_cell1,load_cell2,load_cell3,esmode1_cell0,esmode1_cell1,esmode1_cell2,esmode1_cell3,...,day,hour,energy,split,energy_1,energy_2,energy_3,energy_4,energy_5,residual
0,2023-01-01 01:00:00,B_0,0.487936,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,1,1,64.275037,train,NaN,NaN,NaN,NaN,NaN,58.590689
1,2023-01-01 01:00:00,B_1,0.034770,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,1,1,18.086697,train,NaN,NaN,NaN,NaN,NaN,19.379589


In [ ]:
from sklearn import metrics
from sklearn import model_selection
import lightgbm as lgb
import xgboost as xgb
import numpy as np
kf = model_selection.KFold(n_splits=5)
kf = kf.split(X=train_KPI_merge_KPI_lagging)

oof_valid_preds = np.zeros(train_KPI_merge_KPI_lagging.shape[0], )
test_preds_list = []

for i, (train_idx, valid_idx) in enumerate(kf):

    train_df = train_KPI_merge_KPI_lagging.iloc[train_idx]
    valid_df = train_KPI_merge_KPI_lagging.iloc[valid_idx]
    test_df = test_KPI_merge_KPI_lagging.copy()

    train_df, valid_df, test_df, _ = feature_enginning(train_df=train_df, valid_df=valid_df, test_df=test_df)
    train_cols = [col for col in train_df.columns if col not in drop_cols]

    X_train, y_train = train_df[train_cols], train_df[target_col]
    X_valid, y_valid = valid_df[train_cols], valid_df[target_col]
    X_test = test_df[train_cols]

    params = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'max_depth' : -1,
        'metric': 'mape',
        'num_boost_round': 10000,
    }

    # model = lgb.LGBMRegressor(**lgb_study.best_params)
    model2 = lgb.LGBMRegressor(**params)
    # model = lgb.LGBMRegressor()

    early_stopping_callback = lgb.early_stopping(100, first_metric_only=True, verbose=False)
    verbose_callback = lgb.log_evaluation(100)

    model2.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[early_stopping_callback, verbose_callback],
    )

    valid_preds = model2.predict(X_valid)
    test_preds = model2.predict(X_test)

    val_score = metrics.mean_absolute_percentage_error(y_valid, valid_preds)
    oof_valid_preds[valid_idx] = valid_preds
    test_preds_list.append(test_preds)

    print("=*"*50)
    print(f"Fold : {i}")
    print(f"Valid score : ", val_score)

oof_score = metrics.mean_absolute_percentage_error(train_KPI_merge_KPI_lagging[target_col], oof_valid_preds)
print("_-*"*50)
print(f"OOF score : ", oof_score)

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047829 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8307
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 139
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.313705
[100]	valid_0's mape: 0.111331
[200]	valid_0's mape: 0.104155
[300]	valid_0's mape: 0.0988641
[400]	valid_0's mape: 0.0953309
[500]	valid_0's mape: 0.0908592
[600]	valid_0's mape: 0.089683
[700]	valid_0's mape: 0.0879213
[800]	valid_0's mape: 0.0861236
[900]	valid_0's mape: 0.0854619
[1000]	valid_0's mape: 0.0849169
[1100]	valid_0's mape

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7709
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 73
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.276547
[100]	valid_0's mape: 0.0782767
[200]	valid_0's mape: 0.0732549
[300]	valid_0's mape: 0.0698422
[400]	valid_0's mape: 0.0680305
[500]	valid_0's mape: 0.0668625
[600]	valid_0's mape: 0.0663172
[700]	valid_0's mape: 0.0655773
[800]	valid_0's mape: 0.0649638
[900]	valid_0's mape: 0.0641687
[1000]	valid_0's mape: 0.0634411
[1100]	valid_0's ma

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.172555 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8388
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 139
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.059540
[100]	valid_0's mape: 0.0637756
[200]	valid_0's mape: 0.0602644
[300]	valid_0's mape: 0.0584122
[400]	valid_0's mape: 0.0571102
[500]	valid_0's mape: 0.0562282
[600]	valid_0's mape: 0.0554365
[700]	valid_0's mape: 0.0551432
[800]	valid_0's mape: 0.0547347
[900]	valid_0's mape: 0.0544499
[1000]	valid_0's mape: 0.0540649
[1100]	valid_0's m

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047237 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8382
[LightGBM] [Info] Number of data points in the train set: 74103, number of used features: 139
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 27.913267
[100]	valid_0's mape: 0.0619732
[200]	valid_0's mape: 0.0590199
[300]	valid_0's mape: 0.0566381
[400]	valid_0's mape: 0.0552742
[500]	valid_0's mape: 0.0541037
[600]	valid_0's mape: 0.0535033
[700]	valid_0's mape: 0.0528276
[800]	valid_0's mape: 0.0524812
[900]	valid_0's mape: 0.0520757
[1000]	valid_0's mape: 0.0518328
[1100]	valid_0's m

<ipython-input-16-5d20585fe547>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'
<ipython-input-16-5d20585fe547>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['split'] = 'valid'
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052378 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8070
[LightGBM] [Info] Number of data points in the train set: 74104, number of used features: 137
[LightGBM] [Warning] num_iterations is set=10000, num_boost_round=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Info] Start training from score 28.131925
[100]	valid_0's mape: 0.0556199
[200]	valid_0's mape: 0.0522879
[300]	valid_0's mape: 0.050776
[400]	valid_0's mape: 0.0497012
[500]	valid_0's mape: 0.0491696
[600]	valid_0's mape: 0.0486215
[700]	valid_0's mape: 0.048374
[800]	valid_0's mape: 0.0482215
[900]	valid_0's mape: 0.0480951
[1000]	valid_0's mape: 0.0479196
[1100]	valid_0's map

In [ ]:
test_preds_mean = np.mean(test_preds_list, axis=0)

In [ ]:
submission = pd.DataFrame()
submission['ID'] = test_df['time'] + '_' + test_df['bs']

In [ ]:
submission['Energy'] = test_preds_mean

In [ ]:
submission.to_csv("submission.csv", index=False)